In [1]:
import streamlit as st
import pandas as pd
import ast
from ast import literal_eval
import chromadb
from chromadb.utils import embedding_functions
import openai
from pathlib import Path
import base64
import uuid
from openai import AzureOpenAI
from chromadb import Documents, EmbeddingFunction, Embeddings

In [2]:
def retreive_relevant_docs(chromdb_collection,prompt_embeddings):
    title_query_result = chromdb_collection.query(
        query_embeddings=prompt_embeddings,
        n_results=10,
        include=['metadatas','distances', 'documents']
    )
#     print(title_query_result)
    result = pd.DataFrame({
                'id':title_query_result['ids'][0], 
                'score':title_query_result['distances'][0],
                'docs':title_query_result['documents'][0],
                'metadata': title_query_result['metadatas'][0]
                })
    return result

In [3]:
def get_embeddings(input_text):
    client = AzureOpenAI(
    api_key = "3a3b78172a564e5187901b0b2a268100",
    api_version = "2023-05-15",
    azure_endpoint = "https://usammcomponentfactoryopenaiservicetmna.openai.azure.com/"
    )
    
    response = client.embeddings.create(
        input = input_text,
        model= "TextEmbeddingADA002"
    )
    
    return response.data[0].embedding

In [4]:
def get_response(prompt, system, temperature = 1.0):
    client = AzureOpenAI(
    api_key = "3a3b78172a564e5187901b0b2a268100",  
    api_version = "2023-05-15",
    azure_endpoint = "https://usammcomponentfactoryopenaiservicetmna.openai.azure.com/"
    )

    response = client.chat.completions.create(
        model="TurboCodeGPT35Turbo", # model = "deployment_name".
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        # max_tokens=3000
    )

    #print(response)
    # print(response.model_dump_json(indent=2))
    return response.choices[0].message.content

In [5]:
def create_context(df):
    context = ''
    # print(df['documents'])
    for i, row in df[['metadata', 'docs']].iterrows():
        
        context += f"Program Name. {row['metadata']['program_name']} ,Rule Sequence no. {row['metadata']['Rule_Sequence_Number']} :\n{row['docs']}\n"
        
#     print('context: ', context)
    return context

In [6]:
# dtype = {'Code/Database': str, 'Functional Description':str,'English Translation':str}
df = pd.read_csv('PASB400_annotations_refined.csv')
df = df.drop_duplicates()

In [7]:
df

,Annotation Type,Category,State,Source Code,Rule Sequence Number,Annotation Description,Code/Database,Functional Description,Modified By,English Translation
0,RULE,Technical Rule,CANDIDATE,EXEC SQL ...,0.1,Declare C1-DENVCAL cursor:\nThis code declares...,NaN,NaN,Subha Murugan,NaN
1,RULE,Technical Rule,CANDIDATE,EXEC SQL ...,0.2,Declare C2-MDSCPCO:\nThis code is declaring a ...,NaN,NaN,Subha Murugan,NaN
2,RULE,Technical Rule,CANDIDATE,EXEC SQL ...,0.3,Declare cursor C3-MDSITCR:\nThis code declares...,NaN,NaN,Subha Murugan,NaN
3,RULE,Technical Rule,CANDIDATE,IF WS-IN01-STATUS = '00' ...,1.1,File status check on opening input file 01:\n\...,NaN,NaN,Subha Murugan,NaN
4,RULE,Technical Rule,CANDIDATE,IF WS-IN02-STATUS = '00' ...,1.2,File status check on opening input file 02:\nT...,NaN,NaN,Subha Murugan,NaN
...,...,...,...,...,...,...,...,...,...,...
69,RULE,Process Rule,CANDIDATE,EVALUATE SQLCODE ...,13.2,Evaluate the SQL code for close cursor C1-DENV...,NaN,NaN,Subha Murugan,NaN
70,RULE,Error Processing Rule,CANDIDATE,WHEN OTHER ...,13.3,Evaluate the SQL code for close cursor C1-DENV...,NaN,NaN,Subha Murugan,NaN
71,RULE,Technical Rule,CANDIDATE,IF WS-IN01-STATUS = '00' ...,13.4,This code checks if the status of a file named...,NaN,NaN,Subha Murugan,NaN
72,RULE,Technical Rule,CANDIDATE,IF WS-IN02-STATUS = '00' ...,13.5,This code checks if the status of a file named...,NaN,NaN,Subha Murugan,NaN


In [10]:
df = df.rename(columns = {'Functional Description':'Functional_Description'})
df = df.rename(columns = {'Rule Sequence Number':'Rule_Sequence_Number'})
df = df.rename(columns = {'Annotation Description':'Annotation_Description'})

In [11]:
embeddings = []
for i in range(len(df['Functional_Description'])):
    embd = get_embeddings(str(df['Functional_Description'][i]))
    embeddings.append(embd)

In [12]:
df['embeddings'] = embeddings

In [29]:
df['program_name'] = 'PASB400'

In [30]:
df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]

In [31]:
df['Functional_Description'] = df['Functional_Description'].astype(str)
# df['embeddings'] = df['embeddings'].apply(literal_eval)

In [33]:
df['embeddings']

0     [0.0018096872372552752, -0.02556374855339527, ...
1     [0.0018096872372552752, -0.02556374855339527, ...
2     [0.0018096872372552752, -0.02556374855339527, ...
3     [0.0018096872372552752, -0.02556374855339527, ...
4     [0.0018096872372552752, -0.02556374855339527, ...
                            ...                        
69    [0.0018096872372552752, -0.02556374855339527, ...
70    [0.0018096872372552752, -0.02556374855339527, ...
71    [0.0018096872372552752, -0.02556374855339527, ...
72    [0.0018096872372552752, -0.02556374855339527, ...
73    [0.0018096872372552752, -0.02556374855339527, ...
Name: embeddings, Length: 74, dtype: object

In [34]:
result_df = df[['program_name','Source Code','Rule_Sequence_Number','Annotation_Description','Functional_Description', 'embeddings','id']]
result_df['id'] = result_df['id'].apply(str)

C:\Users\ayussahu\AppData\Local\Temp\ipykernel_12880\2147554999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['id'] = result_df['id'].apply(str)


In [35]:
result_df['Rule_Sequence_Number'] = result_df['Rule_Sequence_Number'].apply(lambda x: {'Rule_Sequence_Number' : x})

C:\Users\ayussahu\AppData\Local\Temp\ipykernel_12880\1269197329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Rule_Sequence_Number'] = result_df['Rule_Sequence_Number'].apply(lambda x: {'Rule_Sequence_Number' : x})


In [36]:
result_df['program_name'] = result_df['program_name'].apply(lambda x: {'Program_name' : x})

C:\Users\ayussahu\AppData\Local\Temp\ipykernel_12880\3120863514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['program_name'] = result_df['program_name'].apply(lambda x: {'Program_name' : x})


In [37]:
chroma_client = chromadb.Client()
chromdb_collection = chroma_client.get_or_create_collection(name='stories')

In [38]:
metadf = df[['program_name','Source Code','Rule_Sequence_Number','Annotation_Description']]
data = metadf.to_dict(orient='records')

In [39]:
chromdb_collection.add(
    ids=result_df.id.tolist(),
    embeddings=result_df.embeddings.tolist(),
    metadatas=data,
    documents=result_df.Functional_Description.tolist()
    )

In [40]:
try:
    # Get text input from the user
    user_query = None
    user_query_input = str(input("Enter some text:"))

    # Check if there is no input
    if user_query_input:
        user_query = user_query_input
        # print('inside 1111111111111')
    elif prompt1: 
        user_query = 'How is SSN validated ?'
        # print('inside 222222222222222')

    elif prompt2:
        user_query = "List all business rules related to production planning"
        # print('inside 33333333333333333')

    elif prompt3:
        user_query = "List all business rules related to production periods"
        # print('inside 444444444444444')

    if user_query:
#         print('*************user query:*************** ', user_query)
        title_query_result = retreive_relevant_docs(chromdb_collection, get_embeddings(user_query))
#         print('*************relevant documents**************')
#         print(title_query_result)
        # st.write(title_query_result['documents'])
        context = create_context(title_query_result)
        # print(context)

        # generate RAG responses by passing top relevant documents
        system_prompt = """You are an intelligent question answer assistant. 
        Your task is to display business rules,Program name and Rule Sequence no from the provided context based on the questions asked by user. 
        Display accuracy score of your answer at the end."""

        message_prompt = f"""
        Question:
        ```
        {user_query}
        ```

        Context:
        ```
        {context}
        ```

        Answer:

        """

        response = get_response(message_prompt, system_prompt, temperature = 0.8)

        print(response)
        
except Exception as e:
    print(e)

Enter some text:list all business rules related to production year
Business rules related to production year:

1) Program Name: PASB400, Rule Sequence no: 5.6
   - If the production year and month of sales PPO forecast file is equal to the Period1 production year and month of DENVCAL table, perform certain actions.

2) Program Name: PASB400, Rule Sequence no: 5.7
   - If the production year and month of sales PPO forecast file is equal to the Period2 or Period3 or Period4 production year and month of DENVCAL table, perform certain actions.

3) Program Name: PASB400, Rule Sequence no: 4.2
   - If the production period is 'PPR0', use Period1 of DENVCAL fetch for processing.

4) Program Name: PASB400, Rule Sequence no: 4.3
   - If the production period is 'PPR1', use Period2, Period3, and Period4 of DENVCAL fetch for processing.


Accuracy Score: 100%
